In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
import os
import sys
sys.path.append('..')
import visualization
from visualization.gradcam import GradCam
from visualization import misc_functions
from visualization.misc_functions import get_params, save_class_activation_on_image, show_class_activation_on_image

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = '../data/'
test_dir = 'normal_test'
classes = sorted(os.listdir(data_dir + test_dir))
batch_size = 8

In [19]:
mean = [0.44947562, 0.46524084, 0.40037745]
std = [0.18456618, 0.16353698, 0.20014246]
#mean = [0.14318287, 0.19182085, 0.10939839]
#std = [0.20657195, 0.25984347, 0.1585114]

data_transforms = {
        'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)])}

test_images = datasets.ImageFolder(os.path.join(data_dir, test_dir),
                    data_transforms['test'])

test_dataloader = DataLoader(test_images, batch_size=batch_size, shuffle=False, num_workers=4)

In [10]:
model = models.AlexNet()
model.classifier[6] = nn.Linear(4096, 10)
model.load_state_dict(torch.load('alexnet_pretrained.model', map_location=str(device)))
model.eval()
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [ ]:
def image_show(img):
    img = np.asarray(std).mean() * img + np.asarray(mean).mean()
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.pause(0.001)

In [ ]:
file_name_to_export = "prova"
grad_cam = GradCam(model, target_layer=11)
model.train()
#with torch.no_grad():
for data in test_dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    for i in range(len(inputs)):
        if(labels[i] == 2):
            original = np.transpose(inputs[i].numpy(),(1,2,0))
            single_input = inputs[i].unsqueeze_(0)
            cam = grad_cam.generate_cam(single_input, labels[i])
            print(classes[labels[i]])
            image_show(original)
            show_class_activation_on_image(original, cam)
            #save_class_activation_on_image(np.transpose(inputs[i].numpy(),(1,2,0)), cam, file_name_to_export)
        
        #cam = grad_cam.generate_cam(inputs, labels)
        #save_class_activation_on_image(original_image, cam, file_name_to_export)
